# Step 3: Virtualize bulk data through an external service

In this final version of our story, we will again be writing a Table with manually created bulk data Urls.
But instead of writing the bulk data to files, we will only write a lookup table, which will then be used by an
external HTTP server for serving chunks dynamically.

In this solution we will generate a single Url for every single bulk data attribute. This makes it easier to resolve Urls to data.

In [ ]:
import numpy as np
import tlc
from data_sources import Deterministic3DPointCloudDataset

In [ ]:
server_root = "http://localhost:2233"

In [ ]:
dataset = Deterministic3DPointCloudDataset(10)

In [ ]:
rows = []

for i in range(len(dataset)):
    data = dataset[i]
    intensities: np.ndarray = data["intensities"]
    vertices_3d: np.ndarray = data["vertices_3d"]
    vertices_3d_length = np.prod(vertices_3d.shape) * vertices_3d.dtype.itemsize
    intensities_length = np.prod(intensities.shape) * intensities.dtype.itemsize

    row = {
        "x_min": 0,
        "y_min": 0,
        "z_min": 0,
        "x_max": 1,
        "y_max": 1,
        "z_max": 1,
        "instances": [
            {
                "vertices_3d_binary_property_url": f"{server_root}/sample-{i}-vertices_3d.raw:0-{vertices_3d_length}",
                "vertices_3d_additional_data": {
                    "intensity_binary_property_url": f"{server_root}/sample-{i}-intensities.raw:0-{intensities_length}"
                },
            }
        ],
    }

    rows.append(row)

## Write the Table!

In [ ]:
schema = tlc.Geometry3DSchema(
    include_3d_vertices=True,
    per_vertex_schemas={"intensity": tlc.Float32ListSchema()},
    is_bulk_data=True,  # This is what sets up the "sibling" paths with the "_binary_property_url" suffix
)

table_writer = tlc.TableWriter(
    table_name="externalize-virtual-http-endpoints-single-attrs",
    dataset_name="pre-externalized-dataset",
    project_name="External Bulk Data",
    description="Pre-externalized table",
    column_schemas={"vertices": schema},
    if_exists="rename",
)

In [ ]:
for row in rows:
    table_writer.add_row({"vertices": row})

table = table_writer.finalize()

In [ ]:
table[0]

Run the corresponding HTTP-server `python http_server_single_attrs.py` before accessing this Table in the Dashboard.